In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from scipy.stats import rankdata
import pandas as pd
%matplotlib inline

In [ ]:
f = h5py.File('test_data.h5')


In [ ]:
p = h5py.File('../../../../predictions/plants/ensemble_IiFEB_IiFEB_11_TSTA8_TSTA8_6_m5MV8_m5MV8_11_mjnps_mjnps_6/Mesculenta/predictions.h5')

In [ ]:
f['data/X'].shape[0]

In [ ]:
# select examples of interest
# just look at first 10k chunks to avoid mem errors and speed things up
enough = 10_000

# mask to skip anything with padding
unpadded = np.sum(np.sum(f['data/X'][:enough], axis=2), axis=1) == 20000

In [ ]:
# mask to skip anything that's just big and totally ig
anynotig = np.argmax(f['data/y'][:enough], axis=2) + np.argmax(p['predictions'][:enough], axis=2) > 0
notig10 = np.sum(anynotig, axis=1) / 20000 > 0.1

# percent accuracy
right = np.argmax(f['data/y'][:enough], axis=2) == np.argmax(p['predictions'][:enough], axis=2)
acc = np.sum(right, axis=1) / 20000

# track ori indexes
#h5i = np.arange(f['data/X'].shape[0])
h5i = np.arange(enough)

In [ ]:
mask = unpadded & notig10
h5i = h5i[mask]
rankacc = rankdata(acc[mask])
rankscore = rankdata(f['scores/one_centered'][:enough][mask])

In [ ]:
upper = h5i.shape[0] * 0.9
lower = h5i.shape[0] * 0.1

#Low accuracy, high ref score
candidates = h5i[(rankacc < lower) & (rankscore > upper)]
#Low accuracy, low ref score
#andidates = h5i[(rankacc < lower) & (rankscore < lower)]
#High accuracy, high ref score
#candidates = h5i[(rankacc > upper) & (rankscore > upper)]

#High accuracy, low ref score
#candidates = h5i[(rankacc > upper) & (rankscore < lower)]
print(len(candidates))
print(candidates)

In [ ]:
h5i.shape

In [ ]:
j = -1


In [ ]:
j += 1
i = candidates[j]
print(i)


print(f['data/seqids'][i], f['data/start_ends'][i])
fig, (ax1, ax2, axaug, ax3) = plt.subplots(4, 1, sharex=True, figsize=(14,7),
                                         gridspec_kw={"height_ratios": [2, 0.5, 0.5, 0.5]})
#plt.figure(figsize=(3,10))

ax1.plot(np.log(f['evaluation/coverage'][i] + 1), c='black')
ax1.plot(np.log(f['evaluation/spliced_coverage'][i] + 1), c='cadetblue')
ax1.set_xlim((0, 20000))
ax1.set_ylabel("ln(coverage + 1)")
ax1.legend(['cov', 'sc'])

yticks = ['', 'i-genic', 'utr', 'cds', 'intron']
ax2.imshow(np.array(f['data/y'][i].T).astype(float), aspect="auto")
ax2.set_yticklabels(yticks)
ax2.set_ylabel('reference')

axaug.imshow(np.array(f['alternative/augustus/y'][i].T).astype(float), aspect="auto")
axaug.set_yticklabels(yticks)
axaug.set_ylabel('augustus')

ax3.imshow(np.array(p['predictions'][i].T).astype(float), aspect="auto")
ax3.set_yticklabels(yticks)
ax3.set_ylabel('helixer')

In [ ]:
def plot_selected(i, f, p, start, end):
    print(i)
    print(f['data/seqids'][i], f['data/start_ends'][i])
    fig, (ax1, ax2, axaug, ax3) = plt.subplots(4, 1, sharex=True, figsize=(7,5),
                                             gridspec_kw={"height_ratios": [1.5, 0.5, 0.5, 0.5]})
    #plt.figure(figsize=(3,10))

    ax1.plot(np.log(f['evaluation/coverage'][i][start:end] + 1), c='black')
    ax1.plot(np.log(f['evaluation/spliced_coverage'][i][start:end] + 1), c='cadetblue')
    ax1.set_xlim((0, end - start))
    ax1.set_ylabel("ln(coverage + 1)")
    #ax1.legend(['cov', 'sc'])

    yticks = ['IG', 'UTR', 'CDS', 'Ntrn']
    ax2.imshow(1 - np.array(f['data/y'][i][start:end].T).astype(float), aspect="auto", cmap='gray')
    ax2.set_ylim([-0.5, 3.5])
    ax2.set_yticks([0,1,2, 3])
    ax2.set_yticklabels(yticks)
    ax2.set_ylabel('Ref.')

    axaug.imshow(1 - np.array(f['alternative/augustus/y'][i][start:end].T).astype(float), aspect="auto", cmap='gray')
    axaug.set_ylim([-0.5, 3.5])
    axaug.set_yticks([0,1,2, 3])
    axaug.set_yticklabels(yticks)
    axaug.set_ylabel('Aug.')

    ax3.imshow(1 - np.array(p['predictions'][i][start:end].T).astype(float), aspect="auto", cmap='gray')
    ax3.set_ylim([-0.5, 3.5])
    ax3.set_yticks([0,1,2, 3])
    ax3.set_yticklabels(yticks)
    ax3.set_ylabel('Helixer')
    se = f['data/start_ends'][i]

    at = [0, 2000, 4000, 6000, 8000, 10000]
    ax3.set_xticks(at)
    offset = f['data/start_ends'][i][0] + start
    if se[1] - se[0] > 0:
        sign = '+'
        ax3.set_xticklabels([int((offset + x)/ 10**3) for x in at])
    else:
        sign = '-'
        ax3.set_xticklabels([int((offset - x)/ 10**3) for x in at])
    ax3.set_xlabel('position on ({}) strand of {} in kbp'.format(sign, f['data/seqids'][i].decode()))
    

In [ ]:
# high score, low acc
i = 6436
start, end = 0, 10000
plot_selected(i, f, p, start, end)
plt.savefig(fname='highScore_lowAcc_001.eps')

i = 5392
start, end = 7000, 17000
plot_selected(i, f, p, start, end)
plt.savefig(fname='highScore_lowAcc_002.eps')

i = 1162
start, end = 6000, 16000
plot_selected(i, f, p, start, end)
plt.savefig(fname='highScore_lowAcc_003.eps')

In [ ]:
# low score, low acc
# looks good
i = 1024
start, end = 10000, 20000
plot_selected(i, f, p, start, end)
plt.savefig(fname='lowScore_lowAcc_001.eps')

i = 392
start, end = 0, 10000
plot_selected(i, f, p, start, end)
plt.savefig(fname='lowScore_lowAcc_002.eps')


# errors to go around
i = 432
start, end = 10000, 20000
plot_selected(i, f, p, start, end)
plt.savefig(fname='lowScore_lowAcc_003.eps')


In [ ]:
# high acc high score

i = 18
start, end = 5000, 15000
plot_selected(i, f, p, start, end)
plt.savefig(fname='highScore_highAcc_001.eps')


i = 7113
start, end = 2500,12500
plot_selected(i, f, p, start, end)
plt.savefig(fname='highScore_highAcc_002.eps')


i = 3233
start, end = 0, 10000
plot_selected(i, f, p, start, end)
plt.savefig(fname='highScore_highAcc_003.eps')


In [ ]:
# high acc, low score

# ghost gene
i = 5569
start, end = 5000, 15000
plot_selected(i, f, p, start, end)
plt.savefig(fname='lowScore_highAcc_001.eps')

# missed intron?
i = 5967
plot_selected(i, f, p, start, end)
plt.savefig(fname='lowScore_highAcc_002.eps')

# spliced read mapping error
i = 9936
start, end = 3000, 13000
plot_selected(i, f, p, start, end)
plt.savefig(fname='lowScore_highAcc_003.eps')
